In [ ]:
# GENERAL IMPORTS

import pandas as pd
import numpy as np
import scipy as sp

import plotly.offline as py
import plotly.graph_objs as go
import plotly
plotly.tools.set_credentials_file(username='lnumerow', api_key='mDhzegrhH0qf8rlFbzOo')
plotly.offline.init_notebook_mode(connected=True)

import ipywidgets

import datetime
import math
import bisect
import time

In [ ]:
datafile = r'''Data\IVE_bidask1min.txt'''

colnames = ['Date', 'Time', 'BidOpen', 'BidHigh', 'BidLow', 'BidClose', 'AskOpen', 'AskLow', 'AskHigh', 'AskClose']
fullpricedata = pd.read_csv(datafile, names=colnames)

fullpricedata['DateTime'] = (fullpricedata['Date']+fullpricedata['Time']).map(lambda x: datetime.datetime(int(x[6:10]), int(x[0:2]), int(x[3:5]), int(x[10:12]), int(x[13:15])))
del fullpricedata['Date']
del fullpricedata['Time']

fullpricedata = fullpricedata[[(dt >= datetime.datetime(dt.year, dt.month, dt. day, 9, 30, 0)) and (dt <= datetime.datetime(dt.year, dt.month, dt. day, 16, 0, 0)) for dt in fullpricedata['DateTime']]].copy()

In [ ]:
pricedata = fullpricedata[fullpricedata['DateTime'] > datetime.datetime(2019,1,1,0,0,0)].copy()
pricedata = pricedata.reset_index()
del pricedata['index']

In [ ]:
def changeTimeframe(pricedata, timeframe_mins):
    newpricedata = pd.DataFrame()
    
    index = 0
    
    while(index < len(pricedata)):
        newcandle = pricedata.iloc[index].copy()
        starttime = newcandle.DateTime
        
        while(True):
            index = index + 1
            if index == len(pricedata):
                break
            
            candle = pricedata.iloc[index].copy()
            
            if candle.DateTime - starttime >= datetime.timedelta(minutes=timeframe_mins):
                break
                
            newcandle.AskClose = candle.AskClose
            newcandle.AskHigh = max(newcandle.AskHigh, candle.AskHigh)
            newcandle.AskLow = min(newcandle.AskLow, candle.AskLow)
            newcandle.BidClose = candle.BidClose
            newcandle.BidHigh = max(newcandle.BidHigh, candle.BidHigh)
            newcandle.BidLow = min(newcandle.BidLow, candle.BidLow)
        
        newpricedata = newpricedata.append(newcandle)
        
    return newpricedata

In [ ]:
class CandlestickChart:
    
    def updateChart(self, placeholder):
        starttime = datetime.datetime.combine(self.startdatewidget.value, datetime.time())
        endtime = datetime.datetime.combine(self.enddatewidget.value, datetime.time()) + datetime.timedelta(days=1)
        timeframe = self.timeframewidget.value
        if starttime >= endtime or timeframe < 1:
            return
        
        fullpricedata = self.fullpricedata
        chartdata = fullpricedata[fullpricedata['DateTime'] >= starttime]
        chartdata = fullpricedata[fullpricedata['DateTime'] < endtime]
        chartdata = changeTimeframe(chartdata, timeframe).copy().reset_index()
        
        pdata = self.chart.data[0]
        pdata.x = [dt.strftime("%y/%m/%d, %H:%M:%S") for dt in chartdata['DateTime']]
        pdata.low = chartdata['BidLow']
        pdata.open = chartdata['BidOpen']
        pdata.close = chartdata['BidClose']
        pdata.high = chartdata['BidHigh']
        
        pricerange = np.max(pdata.high)-np.min(pdata.low)
        
        tdata = self.chart.data[1]
        if self.transactiondata == None:
            tdata.x = []
            tdata.y = []
        else:
            indices = [bisect.bisect_right(pdata.x, dt.strftime("%y/%m/%d, %H:%M:%S"))-1 for dt in self.transactiondata['datetime']]
            x = []
            y = []
            symbols = []
            hovertext = []
            
            for ii in range(len(indices)):
                x.append(pdata.x[indices[ii]])
                if self.transactiondata['shares'][ii] > 0:
                    y.append(pdata.low[indices[ii]]-0.1*pricerange)
                    symbols.append('triangle-up')
                    hovertext.append('Buy ' + str(self.transactiondata['shares'][ii]) + ' at $' + str(self.transactiondata['price'][ii]))
                else:
                    y.append(pdata.high[indices[ii]]+0.1*pricerange)
                    symbols.append('triangle-down')
                    hovertext.append('Sell ' + str(-self.transactiondata['shares'][ii]) + ' at $' + str(self.transactiondata['price'][ii]))
            
            tdata.x = x
            tdata.y = y
            tdata.marker.symbol = symbols
            tdata.hovertext = hovertext
        
        layout = self.chart.layout
        layout.xaxis.tickvals = [ii for ii in range(len(pdata.x)) if 
                                 chartdata['DateTime'][ii].time() == datetime.time(9, 30)]
        xaxisdatetimes = [chartdata['DateTime'][ii] for ii in layout.xaxis.tickvals]
        layout.xaxis.ticktext = [dt.strftime(' %y/%m/%d ') if dt.time() == datetime.time(9, 30)
                                 else dt.strftime(' %H:%M ') for dt in xaxisdatetimes]
    
    def __init__(self, fullpricedata, transactiondata = None):
        self.fullpricedata = fullpricedata
        self.transactiondata = transactiondata
        
        data = [dict(
                 type = 'candlestick',
                 name = 'Price Data'),
                dict(
                 type = 'scatter',
                 mode = 'markers',
                 marker = dict(
                    size=10,
                    color='black'),
                 hoverinfo = 'text',
                 name = 'Transactions')
               ]
        layout = dict(
                title = 'Candlestick Chart',
                xaxis = dict(
                    type = 'category', 
                    categoryorder = 'category ascending',
                    title = dict(
                        text = 'DateTime'
                    )
                ),
                yaxis = dict(
                    title = dict(
                        text = 'Price'
                    )
                ))
        self.chart = go.FigureWidget( data=data, layout=layout)
        
        # DEFINE WIDGETS AND CALLBACK FUNCTIONS
        self.startdatewidget = ipywidgets.DatePicker(
            description='Start Date',
            value=datetime.date(2019,1,2),
            disabled=False
        )
        self.enddatewidget = ipywidgets.DatePicker(
            description='End Date',
            value=datetime.date(2019,1,2),
            disabled=False
        )
        self.timeframewidget = ipywidgets.IntText(
            description='Timeframe (minutes)',
            value=1,
            disabled=False
        )
        self.widgets = [self.startdatewidget, self.enddatewidget, self.timeframewidget]
        for widget in self.widgets:
            widget.observe(self.updateChart, names = 'value')
        
        self.widgetsdisplay = ipywidgets.VBox([ipywidgets.HBox([self.startdatewidget, self.enddatewidget]),
                                        self.timeframewidget])
        
        # DEFINE DISPLAY CONFIGURATION
        self.display = ipywidgets.VBox([self.chart, self.widgetsdisplay])
        
        self.updateChart(0)

In [ ]:
transactiondata = dict(
    shares = [1, -1], 
    price = [100.00, 101.00], 
    datetime = [datetime.datetime(2019,1,2,9,53,0), datetime.datetime(2019,1,2,10,52,0)])

In [ ]:
# All trading strategies should take the current price data and strategy params as an input and output buy/sell and quantity
def basic_trading_strategy(portfolio, data_dict, param_dict):
    threshold = param_dict['threshold']
    scale = param_dict['scale']
    pricedata = data_dict['pricedata']

    for index in range(len(pricedata)):
        row = pricedata.loc[index]
        
        if index == 0:
            last_row = pricedata.loc[0]

        delta = row['BidOpen'] - last_row['BidClose']
        last_row = row

        if delta > threshold:
            quantity = scale*round(100*delta)
            portfolio.buy(quantity, row)
        elif delta < -threshold:
            quantity = scale*round(-100*delta)
            portfolio.sell(quantity, row)
        
        portfolio.update(row)
    
    portfolio.portfolio_history = pd.DataFrame(portfolio.portfolio_history)
    portfolio.transaction_history = pd.DataFrame(portfolio.transaction_history)

In [ ]:
class Portfolio:
    def __init__(self, starting_cash=10000, starting_position=0, price_type='Worst', commission=0):
        self.cash = starting_cash
        self.position = starting_position
        self.commission = commission
        
        self.price_type = price_type
        if self.price_type == 'Best':
            self.ask_price_key = 'AskLow'
            self.bid_price_key = 'BidHigh'
        elif self.price_type == 'Worst':
            self.ask_price_key = 'AskHigh'
            self.bid_price_key = 'BidLow'
        
        self.net_value = 0
        self.portfolio_history = []
        self.transaction_history = []
        
    def update(self, row):
        price = row[self.bid_price_key]
        self.net_value = self.cash + self.position * price
        self.portfolio_history.append(
                {
                    "DateTime": row['DateTime'],
                    "Cash": self.cash,
                    "Position": self.position,
                    "Net Value": self.net_value
                }
        )
        
    def buy(self, quantity, row):
        price = row[self.ask_price_key]
        cost = quantity*price + self.commission
        if cost < self.cash:
            self.cash -= cost
            self.position += quantity
            self.cash = round(self.cash*100)/100
            
            self.transaction_history.append(
                    {
                        "DateTime": row['DateTime'],
                        "Price": price,
                        "Quantity": quantity
                    }
            )
        
    def sell(self, quantity, row):
        price = row[self.bid_price_key]
        if quantity <= self.position:
            self.cash += quantity*price - self.commission
            self.position -= quantity
            self.cash = round(self.cash*100)/100
            
            self.transaction_history.append(
                    {
                        "DateTime": row['DateTime'],
                        "Price": price,
                        "Quantity": -quantity
                    }
            )
        else:
            a=1
            #error('FUCK out of shares')

In [ ]:
param_dict = {'threshold': 0.01, 'scale': 5}
data_dict = {'pricedata': pricedata}
portfolio = Portfolio()
basic_trading_strategy(portfolio, data_dict, param_dict)

In [ ]:
class ProfitCharts():
    def __init__(self, portfolio):
        self.portfolio = portfolio
        cash = portfolio.portfolio_history['Cash']
        position = portfolio.portfolio_history['Position']
        net_value = portfolio.portfolio_history['Net Value']

        data1 = [ dict(
                type = 'scatter',
                x = [ii for ii in range(len(cash))],
                y = cash),

                dict(
                type = 'scatter',
                x = [ii for ii in range(len(cash))],
                y = net_value - cash),

                dict(
                type = 'scatter',
                x = [ii for ii in range(len(cash))],
                y = net_value)
               ]
        
        layout1 = dict(
                title = 'Profit Chart',
                xaxis = dict(
                    title = dict(
                        text = 'Time'
                    )
                ),
                yaxis = dict(
                    title = dict(
                        text = 'Price'
                    )
                ))
        
        self.chart1 = go.FigureWidget( data=data1, layout=layout1)
        
        data2 = [
                dict(
                type = 'scatter',
                x = [ii for ii in range(len(cash))],
                y = [nv - net_value[0] for nv in net_value])
               ]
        
        layout2 = dict(
                title = 'Profit Chart',
                xaxis = dict(
                    title = dict(
                        text = 'Time'
                    )
                ),
                yaxis = dict(
                    title = dict(
                        text = 'Price'
                    )
                ))
        
        self.chart2 = go.FigureWidget( data=data2, layout=layout2)        
        # DEFINE DISPLAY CONFIGURATION
        self.display = ipywidgets.VBox([self.chart1, self.chart2])

In [ ]:
candle = CandlestickChart(pricedata, transactiondata)
candle.display

In [ ]:
profit_charts = ProfitCharts(portfolio)
profit_charts.display